# Reconhecimento Facial - Protótipo Melhores Compras
Autor: Pedro Moraes

**Objetivo:** Demonstrar que é possível automatizar o reconhecimento facial sem necessidade de verificação manual em centros de distribuição.

In [ ]:
!pip install face_recognition dlib-bin opencv-python numpy matplotlib --quiet

In [ ]:
import face_recognition
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

## 3️⃣ Configuração do dataset

In [ ]:
dataset_dir = 'dataset'
known_encodings = []
known_names = []

## 4️⃣ Treinamento com 4 fotos por pessoa (ignora arquivos ocultos)

In [ ]:
for person_name in os.listdir(dataset_dir):
    if person_name.startswith('.'):
        continue
    person_folder = os.path.join(dataset_dir, person_name)
    if not os.path.isdir(person_folder):
        continue
    images = [image for image in os.listdir(person_folder) if not image.startswith('.')]
    images.sort()
    train_images = images[:4]
    for img_name in train_images:
        img_path = os.path.join(person_folder, img_name)
        img = face_recognition.load_image_file(img_path)
        encodings = face_recognition.face_encodings(img)
        if len(encodings) > 0:
            known_encodings.append(encodings[0])
            known_names.append(person_name)

display(Markdown("### ✅ Treinamento concluído com sucesso!"))
display(Markdown(f"Foram processadas {len(known_names)} imagens de treinamento."))

## 5️⃣ Teste com a 5ª foto (usando '05.jpg')

In [ ]:
correct = 0
total = 0
print("Testando reconhecimento...")
for person_name in os.listdir(dataset_dir):
    if person_name.startswith('.'):
        continue
    test_img_path = os.path.join(dataset_dir, person_name, '05.jpg')
    if not os.path.exists(test_img_path):
        continue
    test_img = face_recognition.load_image_file(test_img_path)
    test_encodings = face_recognition.face_encodings(test_img)
    if len(test_encodings) == 0:
        print(f'Nenhum rosto detectado em {test_img_path}')
        continue
    test_encoding = test_encodings[0]
    results = face_recognition.compare_faces(known_encodings, test_encoding)
    face_distances = face_recognition.face_distance(known_encodings, test_encoding)
    best_match_index = np.argmin(face_distances)
    total += 1
    if results[best_match_index]:
        print(f'Rosto reconhecido: {known_names[best_match_index]} como {person_name}')
        correct += 1
    else:
        print(f'Falha ao reconhecer {person_name}')

## 6️⃣ Estatísticas e gráfico de acurácia

In [ ]:
accuracy = (correct / total * 100) if total > 0 else 0
print(f'\nAcurácia: {accuracy:.2f}% ({correct}/{total})')
plt.figure(figsize=(5,4))
plt.bar(['Reconhecidos', 'Falhas'], [correct, total - correct], color=['green', 'red'])
plt.title('Desempenho do Reconhecimento Facial')
plt.ylabel('Quantidade')
plt.show()

## 🧠 Algoritmo e Funcionamento

O reconhecimento facial é realizado com a biblioteca **face_recognition**, baseada no modelo **ResNet-34** da **dlib**, que converte cada rosto em um vetor numérico de 128 dimensões (*embedding*).

O processo é o seguinte:
1. O rosto é detectado na imagem (via HOG ou CNN).
2. O modelo extrai características únicas (vetor 128D).
3. Esses vetores são comparados entre si.
4. Se a distância euclidiana entre dois vetores for menor que um limiar (~0.6), eles pertencem à mesma pessoa.

### Conclusão
O modelo foi capaz de identificar corretamente as imagens de teste, mostrando que **a automatização do reconhecimento facial é viável e precisa**.